In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import anndata as ad
import dynamo as dyn

# Dynamo (gene-based)

In [7]:
adata = ad.read_h5ad("../08_blastocyst/results/blastocyst_counts.gene_based.gene_name.annotated.filtered.h5ad")
adata2 = ad.read_h5ad("../08_blastocyst/results/scanpy_processed_gene_based.h5ad")
adata.obs["CellType"] = adata2.obs["CellType"]
adata.layers["new"] = adata.layers["nascent"]
adata = adata[(adata.obs["s4U"] == 400) & (adata.obs["Time"] == 3) & (adata.obs["ActD"].isna()) & (adata.obs["TC.Ratio"] >= 0)]
adata.write("results/dynamo_data_gene_based.h5ad", compression="gzip")
print(adata)

View of AnnData object with n_obs × n_vars = 2136 × 28794
    obs: 'Run', 'Barcode', 'Species', 's4U', 'Time', 'ActD', 'Group', 'CellLine', 'Stage', 'Platform', 'Label', 'UMI', 'Cells', 'LibStruct', 'Remark', 'Cell.Reads', 'Trimmed.Reads', 'Trimmed.Ratio', 'Mapped.Reads', 'Mapped.Ratio', 'Mito.Ratio', 'Filtered.Reads', 'Filtered.Ratio', 'FilteredClip.Reads', 'FilteredClip.Ratio', 'UMIs', 'UMIs.2Reads', 'Duplicate.Reads', 'Duplicate.Ratio', 'Unique.Reads', 'Genes', 'Isoforms.Assembled', 'Isoforms.Known', 'AC.Ratio', 'AG.Ratio', 'AT.Ratio', 'CA.Ratio', 'CG.Ratio', 'CT.Ratio', 'GA.Ratio', 'GC.Ratio', 'GT.Ratio', 'TA.Ratio', 'TC.Ratio', 'TG.Ratio', 'Pe', 'Pc', 'PcPe.Ratio', 'UMIs.2Reads.Nascent.2TCs', 'UMIs.2Reads.Nascent.2TCs.Ratio', 'Genes.2Reads', 'Genes.2Reads.Nascent.2TCs', 'CellType'
    var: 'GeneID', 'GeneType', 'Chrom', 'Start', 'End', 'Strand'
    layers: 'nascent', 'total', 'new'


In [14]:
adata = ad.read_h5ad("results/dynamo_data_gene_based.h5ad")
dyn.pp.recipe_monocle(adata, tkey="Time")
dyn.tl.dynamics(adata, model='stochastic', cores=3, NTR_vel=True)
dyn.tl.reduceDimension(adata)
dyn.tl.leiden(adata)
dyn.tl.hdbscan(adata)
dyn.tl.louvain(adata)
if True:
    lineage_dict = {"TE-1": ["muralTE", "polarTE"], "ICM-1": ["EPI", "PE"]}
    dyn.tl.gene_wise_confidence(adata, group='CellType', lineage_dict=lineage_dict, ekey="M_t", vkey="velocity_T")
    adata = adata[:,(adata.var["avg_prog_confidence"] > 0) & (adata.var["avg_mature_confidence"] > 0.5)]
dyn.tl.cell_velocities(adata, calc_rnd_vel=True)
dyn.vf.VectorField(adata, basis='umap')

|-----? dynamo.preprocessing.deprecated is deprecated.
|-----> recipe_monocle_keep_filtered_cells_key is None. Using default value from DynamoAdataConfig: recipe_monocle_keep_filtered_cells_key=True
|-----> recipe_monocle_keep_filtered_genes_key is None. Using default value from DynamoAdataConfig: recipe_monocle_keep_filtered_genes_key=True
|-----> recipe_monocle_keep_raw_layers_key is None. Using default value from DynamoAdataConfig: recipe_monocle_keep_raw_layers_key=True
|-----> apply Monocole recipe to adata...


/home/chenzonggui/tmp/ipykernel_12844/1399744631.py:2: DeprecationWarning: recipe_monocle is deprecated and will be removed in a future release. Please update your code to use the new replacement function.
  dyn.pp.recipe_monocle(adata, tkey="Time")


|-----> ensure all cell and variable names unique.
|-----> ensure all data in different layers in csr sparse matrix format.
|-----> ensure all labeling data properly collapased
|-----> detected experiment type: one-shot
|-----> filtering cells...
|-----> 2136 cells passed basic filters.
|-----> filtering gene...
|-----> 13179 genes passed basic filters.
|-----> calculating size factor...
|-----> selecting genes in layer: X, sort method: SVR...
|-----> size factor normalizing the data, followed by log1p transformation.
|-----> Set <adata.X> to normalized data
|-----> applying PCA ...
|-----> <insert> X_pca to obsm in AnnData Object.
|-----> cell cycle scoring...
|-----> computing cell phase...


/home/chenzonggui/miniconda3/envs/py3/lib/python3.10/site-packages/dynamo/preprocessing/utils.py:730: RuntimeWarning: invalid value encountered in divide
  var_ntr = adata.layers["new"].sum(0) / adata.layers["total"].sum(0)


|-----> [Cell Phase Estimation] completed [100.4653s]
|-----> [Cell Cycle Scores Estimation] completed [0.2312s]
|-----> [recipe_monocle preprocess] completed [27.1470s]
|-----> dynamics_del_2nd_moments_key is None. Using default value from DynamoAdataConfig: dynamics_del_2nd_moments_key=False
|-----------> removing existing M layers:[]...
|-----------> making adata smooth...
|-----> calculating first/second moments...
|-----> [moments calculation] completed [20.0558s]
|-----> retrieve data for non-linear dimension reduction...
|-----> [UMAP] using X_pca with n_pca_components = 30
|-----> <insert> X_umap to obsm in AnnData Object.
|-----> [UMAP] completed [25.6490s]
|-----> accessing adj_matrix_key=connectivities built from args for clustering...
|-----> Start computing neighbor graph...
|-----------> X_data is None, fetching or recomputing...
|-----> fetching X data from layer:None, basis:pca
|-----> method arg is None, choosing methods automatically...
|-----------> method ball_tree 

calculating gene velocity vectors confidence based on phase portrait location with priors of progenitor/mature cell types: 2000it [00:43, 45.60it/s]
/home/chenzonggui/miniconda3/envs/py3/lib/python3.10/site-packages/dynamo/tools/metric_velocity.py:436: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  avg = confidence.groupby("gene")["prog_confidence", "mature_confidence"].mean()
/home/chenzonggui/miniconda3/envs/py3/lib/python3.10/site-packages/dynamo/tools/utils.py:2307: ImplicitModificationWarning: Trying to modify attribute `.var` of view, initializing view as actual.
  adata.var[store_key] = (


|-----> [calculating transition matrix via pearson kernel with sqrt transform.] in progress: 100.0000%|-----> [calculating transition matrix via pearson kernel with sqrt transform.] completed [8.9393s]
|-----> [projecting velocity vector to low dimensional embedding] in progress: 100.0000%|-----> [projecting velocity vector to low dimensional embedding] completed [0.9486s]
|-----> [calculating transition matrix via pearson kernel with sqrt transform.] in progress: 100.0000%|-----> [calculating transition matrix via pearson kernel with sqrt transform.] completed [8.8511s]
|-----> [projecting velocity vector to low dimensional embedding] in progress: 100.0000%|-----> [projecting velocity vector to low dimensional embedding] completed [0.9685s]
|-----> VectorField reconstruction begins...
|-----> Retrieve X and V based on basis: UMAP. 
        Vector field will be learned in the UMAP space.
|-----> Generating high dimensional grids and convert into a row matrix.
|-----> Learning vector fi

In [15]:
cmap = plt.colormaps["tab20"]
palette = {
    "ICM-1": cmap(1), 
    "ICM-2": cmap(3),
    "EPI": cmap(5),
    "PE": cmap(7),
    "TE-1": cmap(9),
    "TE-2": cmap(11),
    "muralTE": cmap(13),
    "polarTE": cmap(15)
}
color = "CellType"
pointsize = 0.2
fig, axs = plt.subplots(1, 2, figsize=(7.5, 3))
ax1, ax2 = axs
dyn.pl.streamline_plot(adata, color=color, color_key=palette, pointsize=pointsize, ax=ax1, basis='umap', save_show_or_return='return', show_legend=False)
dyn.pl.streamline_plot(adata, color=color, color_key=palette, pointsize=pointsize, ax=ax2, basis='umap_rnd', save_show_or_return='return', show_legend="right")
ax1.set_title("Gene-based")
ax2.set_title("Gene-based (randomized)")
plt.tight_layout()
if False:
    plt.show()
else:
    plt.savefig("figures/streamline_plot_gene_based.png", dpi=300)
plt.close()

|-----------> plotting with basis key=X_umap
|-----------> skip filtering CellType by stack threshold when stacking color because it is not a numeric type
|-----------> plotting with basis key=X_umap_rnd
|-----------> skip filtering CellType by stack threshold when stacking color because it is not a numeric type


# Dynamo from scanpy results

In [16]:
adata = ad.read_h5ad("../08_blastocyst/results/scanpy_processed_gene_based.h5ad")
adata.layers["new"] = adata.layers["nascent"]
adata.write("results/dynamo_data_gene_based.scanpy_umap.h5ad", compression="gzip")
print(adata)

AnnData object with n_obs × n_vars = 2535 × 28794
    obs: 'Run', 'Barcode', 'Species', 's4U', 'Time', 'ActD', 'Group', 'CellLine', 'Stage', 'Platform', 'Label', 'UMI', 'Cells', 'LibStruct', 'Remark', 'Cell.Reads', 'Trimmed.Reads', 'Trimmed.Ratio', 'Mapped.Reads', 'Mapped.Ratio', 'Mito.Ratio', 'Filtered.Reads', 'Filtered.Ratio', 'FilteredClip.Reads', 'FilteredClip.Ratio', 'UMIs', 'UMIs.2Reads', 'Duplicate.Reads', 'Duplicate.Ratio', 'Unique.Reads', 'Genes', 'Isoforms.Assembled', 'Isoforms.Known', 'AC.Ratio', 'AG.Ratio', 'AT.Ratio', 'CA.Ratio', 'CG.Ratio', 'CT.Ratio', 'GA.Ratio', 'GC.Ratio', 'GT.Ratio', 'TA.Ratio', 'TC.Ratio', 'TG.Ratio', 'Pe', 'Pc', 'PcPe.Ratio', 'UMIs.2Reads.Nascent.2TCs', 'UMIs.2Reads.Nascent.2TCs.Ratio', 'Genes.2Reads', 'Genes.2Reads.Nascent.2TCs', 'leiden', 'Stage_str', 's4U_str', 'ActD_str', 'CellType'
    var: 'GeneID', 'GeneType', 'Chrom', 'Start', 'End', 'Strand', 'highly_variable', 'means', 'dispersions', 'dispersions_norm'
    uns: 'ActD_str_colors', 'CellType

In [17]:
adata = ad.read_h5ad("results/dynamo_data_gene_based.scanpy_umap.h5ad")
dyn.pp.recipe_monocle(adata, tkey="Time")
dyn.tl.dynamics(adata, model='stochastic', cores=3, NTR_vel=True)
dyn.tl.reduceDimension(adata)
dyn.tl.leiden(adata)
dyn.tl.hdbscan(adata)
dyn.tl.louvain(adata)
if True:
    lineage_dict = {"TE-1": ["muralTE", "polarTE"], "ICM-1": ["EPI", "PE"]}
    dyn.tl.gene_wise_confidence(adata, group='CellType', lineage_dict=lineage_dict, ekey="M_t", vkey="velocity_T")
    adata = adata[:,(adata.var["avg_prog_confidence"] > 0) & (adata.var["avg_mature_confidence"] > 0.5)]
dyn.tl.cell_velocities(adata, calc_rnd_vel=True)
dyn.vf.VectorField(adata, basis='umap')

|-----? dynamo.preprocessing.deprecated is deprecated.
|-----> recipe_monocle_keep_filtered_cells_key is None. Using default value from DynamoAdataConfig: recipe_monocle_keep_filtered_cells_key=True
|-----> recipe_monocle_keep_filtered_genes_key is None. Using default value from DynamoAdataConfig: recipe_monocle_keep_filtered_genes_key=True
|-----> recipe_monocle_keep_raw_layers_key is None. Using default value from DynamoAdataConfig: recipe_monocle_keep_raw_layers_key=True
|-----> apply Monocole recipe to adata...


/home/chenzonggui/tmp/ipykernel_12844/1292949160.py:2: DeprecationWarning: recipe_monocle is deprecated and will be removed in a future release. Please update your code to use the new replacement function.
  dyn.pp.recipe_monocle(adata, tkey="Time")


|-----> ensure all cell and variable names unique.
|-----> ensure all data in different layers in csr sparse matrix format.
|-----> ensure all labeling data properly collapased
|-----> detected experiment type: one-shot
|-----> filtering cells...
|-----> 2535 cells passed basic filters.
|-----> filtering gene...
|-----> 13284 genes passed basic filters.
|-----> calculating size factor...
|-----> selecting genes in layer: X, sort method: SVR...
|-----> size factor normalizing the data, followed by log1p transformation.
|-----> Set <adata.X> to normalized data
|-----> applying PCA ...
|-----> <insert> X_pca to obsm in AnnData Object.
|-----> cell cycle scoring...
|-----> computing cell phase...


/home/chenzonggui/miniconda3/envs/py3/lib/python3.10/site-packages/dynamo/preprocessing/utils.py:730: RuntimeWarning: invalid value encountered in divide
  var_ntr = adata.layers["new"].sum(0) / adata.layers["total"].sum(0)


|-----> [Cell Phase Estimation] completed [292.4288s]
|-----> [Cell Cycle Scores Estimation] completed [0.2980s]
|-----> [recipe_monocle preprocess] completed [37.9278s]
|-----> dynamics_del_2nd_moments_key is None. Using default value from DynamoAdataConfig: dynamics_del_2nd_moments_key=False
|-----------> removing existing M layers:[]...
|-----------> making adata smooth...
|-----> calculating first/second moments...
|-----> [moments calculation] completed [23.9304s]
|-----> retrieve data for non-linear dimension reduction...
|-----? adata already have basis umap. dimension reduction umap will be skipped! 
set enforce=True to re-performing dimension reduction.
|-----> [UMAP] completed [0.0043s]
|-----> accessing adj_matrix_key=connectivities built from args for clustering...
|-----> Detecting communities on graph...
|-----------> Converting graph_sparse_matrix to igraph object
|-----> [Community clustering with leiden] completed [401.8837s]
|-----> [hdbscan density-based-clustering] 

calculating gene velocity vectors confidence based on phase portrait location with priors of progenitor/mature cell types: 2000it [00:49, 40.66it/s]
/home/chenzonggui/miniconda3/envs/py3/lib/python3.10/site-packages/dynamo/tools/metric_velocity.py:436: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  avg = confidence.groupby("gene")["prog_confidence", "mature_confidence"].mean()
/home/chenzonggui/miniconda3/envs/py3/lib/python3.10/site-packages/dynamo/tools/utils.py:2307: ImplicitModificationWarning: Trying to modify attribute `.var` of view, initializing view as actual.
  adata.var[store_key] = (


|-----> [calculating transition matrix via pearson kernel with sqrt transform.] in progress: 100.0000%|-----> [calculating transition matrix via pearson kernel with sqrt transform.] completed [5.7226s]
|-----> [projecting velocity vector to low dimensional embedding] in progress: 100.0000%|-----> [projecting velocity vector to low dimensional embedding] completed [1.0654s]
|-----> [calculating transition matrix via pearson kernel with sqrt transform.] in progress: 100.0000%|-----> [calculating transition matrix via pearson kernel with sqrt transform.] completed [5.6963s]
|-----> [projecting velocity vector to low dimensional embedding] in progress: 100.0000%|-----> [projecting velocity vector to low dimensional embedding] completed [1.1009s]
|-----> VectorField reconstruction begins...
|-----> Retrieve X and V based on basis: UMAP. 
        Vector field will be learned in the UMAP space.
|-----> Generating high dimensional grids and convert into a row matrix.
|-----> Learning vector fi

In [18]:
cmap = plt.colormaps["tab20"]
palette = {
    "ICM-1": cmap(1), 
    "ICM-2": cmap(3),
    "EPI": cmap(5),
    "PE": cmap(7),
    "TE-1": cmap(9),
    "TE-2": cmap(11),
    "muralTE": cmap(13),
    "polarTE": cmap(15)
}

color = "CellType"
pointsize = 0.2
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(7.5, 3))
dyn.pl.streamline_plot(adata, color=color, color_key=palette, pointsize=pointsize, ax=ax1, basis='umap', save_show_or_return='return', show_legend=False)
dyn.pl.streamline_plot(adata, color=color, color_key=palette, pointsize=pointsize, ax=ax2, basis='umap_rnd', save_show_or_return='return', show_legend="right")
ax1.set_title("Gene-based")
ax2.set_title("Gene-based (randomized)")
plt.tight_layout()
if False:
    plt.show()
else:
    plt.savefig("figures/streamline_plot_gene_based.scanpy_umap.png", dpi=300)
plt.close()

|-----------> plotting with basis key=X_umap
|-----------> skip filtering CellType by stack threshold when stacking color because it is not a numeric type
|-----------> plotting with basis key=X_umap_rnd
|-----------> skip filtering CellType by stack threshold when stacking color because it is not a numeric type
